# 037. Tokenizer Training from Scratch

In [12]:
!pip install -q KoNLPy
!pip install -U -q sentencepiece

In [2]:
sentences_E = [
    'I love my dog',
    'I love my cat',
    'You love my dog!',
    'I was born in Korea and graduaged University in USA.',
]

sentences_K = [
    "코로나가 심하다",
    "코비드-19가 심하다",
    '아버지가방에들어가신다',
    '아버지가 방에 들어가신다',
    '너무너무너무는 나카무라세이코가 불러 크게 히트한 노래입니다'
]

# 1. Keras 기본 Tokenizer - rule-based
- 공백 또는 구둣점으로 분리  
- 영어 단어별로 띄어쓰기가 철저히 지켜지는 언어

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')  # 빈도수 상위 100 개로 구성

tokenizer.fit_on_texts(sentences_E)           
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'in': 6, 'cat': 7, 'you': 8, 'was': 9, 'born': 10, 'korea': 11, 'and': 12, 'graduaged': 13, 'university': 14, 'usa': 15}


Keras의 rule base tokenizer로 한글을 tokenize

In [6]:
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')  # 빈도수 상위 100 개로 구성

tokenizer.fit_on_texts(sentences_K)           
vocabulary_keras_korean = tokenizer.word_index
print(vocabulary_keras_korean)

{'<OOV>': 1, '심하다': 2, '코로나가': 3, '코비드': 4, '19가': 5, '아버지가방에들어가신다': 6, '아버지가': 7, '방에': 8, '들어가신다': 9, '너무너무너무는': 10, '나카무라세이코가': 11, '불러': 12, '크게': 13, '히트한': 14, '노래입니다': 15}


# 2. 단어 사전 기반 한국어 tokenizer 사용

In [7]:
from konlpy.tag import Okt

okt = Okt()

temp_X = []
for sent in sentences_K:
    temp_X.append(okt.morphs(sent))
    print(okt.morphs(sent))

['코로나', '가', '심하다']
['코', '비드', '-', '19', '가', '심하다']
['아버지', '가방', '에', '들어가신다']
['아버지', '가', '방', '에', '들어가신다']
['너무', '너무', '너', '무', '는', '나카무라', '세이', '코', '가', '불러', '크게', '히트', '한', '노래', '입니다']


사전 기반 tokenize 후 Keras tokenizer 로 vocabulary 생성

In [8]:
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')  # 빈도수 상위 100 개로 구성

tokenizer.fit_on_texts(temp_X)           
vocabulary_okt_keras = tokenizer.word_index
print(vocabulary_okt_keras)

{'<OOV>': 1, '가': 2, '심하다': 3, '코': 4, '아버지': 5, '에': 6, '들어가신다': 7, '너무': 8, '코로나': 9, '비드': 10, '-': 11, '19': 12, '가방': 13, '방': 14, '너': 15, '무': 16, '는': 17, '나카무라': 18, '세이': 19, '불러': 20, '크게': 21, '히트': 22, '한': 23, '노래': 24, '입니다': 25}


두 vocabulary 의 차이 비교

In [9]:
print(vocabulary_keras_korean)
print(vocabulary_okt_keras)

{'<OOV>': 1, '심하다': 2, '코로나가': 3, '코비드': 4, '19가': 5, '아버지가방에들어가신다': 6, '아버지가': 7, '방에': 8, '들어가신다': 9, '너무너무너무는': 10, '나카무라세이코가': 11, '불러': 12, '크게': 13, '히트한': 14, '노래입니다': 15}
{'<OOV>': 1, '가': 2, '심하다': 3, '코': 4, '아버지': 5, '에': 6, '들어가신다': 7, '너무': 8, '코로나': 9, '비드': 10, '-': 11, '19': 12, '가방': 13, '방': 14, '너': 15, '무': 16, '는': 17, '나카무라': 18, '세이': 19, '불러': 20, '크게': 21, '히트': 22, '한': 23, '노래': 24, '입니다': 25}


### 단, Okt 사전에 미등록된 단어의 경우 정확한 tokenizing 이 안된다.

In [10]:
okt.pos('너무너무너무는 나카무라세이코가 불러 크게 히트한 노래입니다')

[('너무', 'Adverb'),
 ('너무', 'Adverb'),
 ('너', 'Modifier'),
 ('무', 'Noun'),
 ('는', 'Josa'),
 ('나카무라', 'Noun'),
 ('세이', 'Noun'),
 ('코', 'Noun'),
 ('가', 'Josa'),
 ('불러', 'Verb'),
 ('크게', 'Noun'),
 ('히트', 'Noun'),
 ('한', 'Josa'),
 ('노래', 'Noun'),
 ('입니다', 'Adjective')]

예를 들어 `너무너무너무`와 `나카무라세이코`는 하나의 단어이지만, okt 사전에 등록되어 있지 않아 여러 개의 복합단어로 나뉘어집니다. 이러한 문제를 해결하기 위하여 형태소 분석기와 품사 판별기들은 사용자 사전 추가 기능을 제공합니다. 사용자 사전을 추가하여 모델의 vocabulary 를 풍부하게 만드는 것은 사용자의 몫입니다.

1. okt 공식 문서를 참고해서 사용사 사전을 추가.
2. okt를 패키징하고, konlpy에서 사용할 수 있도록 konlpy/java 경로에 jar 파일을 복사.
3. 기존에 참고하고 있던 okt.jar 대신 새로운 okt.jar를 사용하도록 설정.
4. konlpy 소스 경로를 import 해서 형태소 분석.

# 3. Google SentencePiece Tokenizer

- NAVER Movie rating data 를 이용한 sentencepiece tokenizer training

In [13]:
import tensorflow as tf
import pandas as pd
import sentencepiece as spm

DATA_TRAIN_PATH = tf.keras.utils.get_file("ratings_train.txt", 
        "https://github.com/ironmanciti/NLP_lecture/raw/master/data/naver_movie/ratings_train.txt")

14794752/14778808 [==============================] - 0s 0us/step


- pandas.read_csv에서 quoting = 3으로 설정해주면 인용구(따옴표)를 무시

In [14]:
train_data = pd.read_csv(DATA_TRAIN_PATH, sep='\t', quoting=3)

print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [15]:
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [16]:
train_data.dropna(inplace=True)

train_data.shape

(149995, 3)

## 학습을 위해 text 를 따로 저장

In [17]:
with open('./nsmc.txt', 'w', encoding='utf-8') as f:
    for line in train_data.document.values:
        try:
            f.write(line + '\n')
        except:
            print("write error ---> ", line)

In [18]:
#write 가 잘 되었는지 확인
with open('./nsmc.txt', 'r', encoding='utf-8') as f:
    nsmc_txt = f.read().split('\n')
    
print(len(nsmc_txt))
print(nsmc_txt[0])

149996
아 더빙.. 진짜 짜증나네요 목소리


In [19]:
input_file = 'nsmc.txt'
vocab_size = 30000
prefix = 'nsmc'

templates = '--input={} --model_prefix={} --vocab_size={}'
cmd = templates.format(input_file, prefix, vocab_size)
cmd

'--input=nsmc.txt --model_prefix=nsmc --vocab_size=30000'

### sentencepiece tokenizer training

In [20]:
spm.SentencePieceTrainer.Train(cmd)

In [21]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(prefix))

True

In [22]:
for t in train_data.document.values[:3]:
    print(t)
    print(sp.encode_as_pieces(t))
    print(sp.encode_as_ids(t), '\n') 

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나네요', '▁목소리']
[52, 752, 5, 25, 16019, 1401] 

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터보고', '▁초딩영화', '줄', '....', '오버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1239, 6, 12819, 18669, 423, 47, 18184, 416, 1133, 6423, 1083, 417] 

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보는것', '을', '추천', '한다']
[17, 600, 21569, 670, 2750, 10787, 14, 2321, 296] 



In [23]:
for line in sentences_K:
    pieces = sp.encode_as_pieces(line)
    ids = sp.encode_as_ids(line)
    print(line)
    print(pieces)
    print(ids)
    print()

코로나가 심하다
['▁코', '로', '나', '가', '▁심하다']
[1359, 29, 33, 13, 5383]

코비드-19가 심하다
['▁코', '비', '드', '-', '19', '가', '▁심하다']
[1359, 334, 277, 282, 3863, 13, 5383]

아버지가방에들어가신다
['▁아버지가', '방', '에', '들어가', '신', '다']
[6161, 618, 16, 13140, 267, 23]

아버지가 방에 들어가신다
['▁아버지가', '▁방', '에', '▁들어가', '신', '다']
[6161, 1569, 16, 3870, 267, 23]

너무너무너무는 나카무라세이코가 불러 크게 히트한 노래입니다
['▁너무너무너무', '는', '▁나카', '무라', '세', '이', '코가', '▁불러', '▁크게', '▁히트', '한', '▁노래', '입니다']
[14344, 12, 17264, 10088, 262, 10, 13095, 3392, 1846, 10169, 30, 765, 228]

